In [ ]:
# Imports
%matplotlib notebook
import matplotlib.pyplot as plt
import sys
from setup import *
from ipywidgets import *
import file_open
import math
print(matplotlib.get_backend())
style = {'description_width': 'initial'}

# Image upload
This notebook processes photos of mayfly eggs and uses blob detection to count the number of eggs in the image. Run the cell below and select a single file to process. The image is converted to greyscale automatically upon uploading. The axes show the image size in pixels to help us select the appropriate size thresholds when setting upi the blob detection algorithm.

fileName = file_open.gui_fname()
im = cv2.imread(fileName.decode("utf-8"), cv2.IMREAD_GRAYSCALE)
h, w = im.shape
aspect_ratio = h/w + 0.1
width = 9.8
fig = plt.figure(figsize=(width,width*aspect_ratio))
plt.subplot(111)
plt.imshow(im, cmap='gray')
plt.title('Greayscale original')
# plt.axis('off')
plt.show()
original = im

# Image thresholding
Firstly, we try to remove all unncesary image artifacts by thresholding the values of each pixel along the greyscale. We will try two types of thresholding: global and adaptive one. In global thresholding, the same threshold value is applied to every pixel in the image. If the pixel value is smaller than the threshold, it is set to 0, otherwise it is set to a maximum value that we define. In the adaptive thresholding, the threshold for each pixel is determined by the small region around it. The first type of thresholding is suitable for images where the lighting is uniform and there are no overexposed areas. The adaptive type is useful for images where the brightness is not uniform. 

thresholdGlobal = 127
pixelNeigbourhood = 11
constant = 4
del thresh_global
ret, thresh_global = cv2.threshold(original,thresh=thresholdGlobal,maxval=255,type=cv2.THRESH_BINARY)
thresh_adapt = cv2.adaptiveThreshold(original, maxValue=255, adaptiveMethod=cv2.ADAPTIVE_THRESH_MEAN_C, thresholdType=cv2.THRESH_BINARY, blockSize=pixelNeigbourhood, C=constant)
# apply morphology open then close to both thresholded images
openSize = 5
closeSize = 9
kernelOpen = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (openSize,openSize))
kernelClose = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (closeSize,closeSize))
openBlob = cv2.morphologyEx(thresh_adapt, cv2.MORPH_OPEN, kernelOpen)
blob_global = cv2.morphologyEx(openBlob, cv2.MORPH_CLOSE, kernelClose)
openBlob = cv2.morphologyEx(thresh_adapt, cv2.MORPH_OPEN, kernelOpen)
blob_adapt = cv2.morphologyEx(openBlob, cv2.MORPH_CLOSE, kernel)
# plot
numCols, numRows = 2, 2
fig, axes = plt.subplots(numRows,numCols, figsize=((width,width*aspect_ratio*numRows/numCols)))
plotter_thresh_g = axes[0,0].imshow(thresh_global, cmap='gray')
axes[0,0].set_title('Global thresholding')
axes[0,0].axis('off')
plotter_blob_g = axes[0,1].imshow(blob_global, cmap='gray')
axes[0,1].set_title('Associated blobs')
axes[0,1].axis('off')
plotter_thresh_a = axes[1,0].imshow(thresh_adapt, cmap='gray')
axes[1,0].set_title('Adaptive thresholding')
axes[1,0].axis('off')
plotter_blob_a = axes[1,1].imshow(blob_adapt, cmap='gray')
axes[1,1].set_title('Associated blobs')
axes[1,1].axis('off')
plt.tight_layout(pad=0.5)
plt.show()
# interactive part
def update_morphology(threshold_upd,block_upd,const_upd,openSize_upd,closeSize_upd):
    ret,thresh_global = cv2.threshold(original,thresh=threshold_upd,maxval=255,type=cv2.THRESH_BINARY)
    thresh_adapt = cv2.adaptiveThreshold(original, maxValue=255, adaptiveMethod=cv2.ADAPTIVE_THRESH_MEAN_C, thresholdType=cv2.THRESH_BINARY, blockSize=block_upd, C=const_upd)
    kernelOpen = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (openSize_upd,openSize_upd))
    kernelClose = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (closeSize_upd,closeSize_upd))
    openBlob = cv2.morphologyEx(thresh_global, cv2.MORPH_OPEN, kernelOpen)
    blob_global = cv2.morphologyEx(openBlob, cv2.MORPH_CLOSE, kernelClose)
    openBlob = cv2.morphologyEx(thresh_adapt, cv2.MORPH_OPEN, kernelOpen)
    blob_adapt = cv2.morphologyEx(openBlob, cv2.MORPH_CLOSE, kernelClose)
    plotter_thresh_g.set_data(thresh_global)
    plotter_blob_g.set_data(blob_global)
    plotter_thresh_a.set_data(thresh_adapt)
    plotter_blob_a.set_data(blob_adapt)
    fig.canvas.draw_idle()
# sliders
thr_slider0 =  widgets.IntSlider(min=1, max=254, step=1, value=thresholdGlobal, description='Global threshold',style=style) 
thr_slider1 =  widgets.IntSlider(min=3, max=200, step=1, value=pixelNeigbourhood, description='Neighbourhood area',style=style) 
thr_slider2 = widgets.IntSlider(min=1, max=100, step=1, value=constant, description='Constant',style=style)
w_slider1 =  widgets.IntSlider(min=3, max=55, step=2, value=open_size, description='Open kernel size',style=style) 
w_slider2 = widgets.IntSlider(min=3, max=55, step=2, value=close_size, description='Close kernel size',style=style)
interact(update_morphology,threshold_upd=thr_slider0, block_upd=thr_slider1,const_upd=thr_slider2,openSize_upd=w_slider1,closeSize_upd=w_slider2);

# Denoising the image
Prior to running the detection algorithm, the image can be denoised to prevent falsely detecting image noise as eggs.
The smoothing procedure "blurres" the photo - we may benifinte from it as it slightly erodes the border of your eggs. Choose one of the methods below:

im = blob_adapt
fig = plt.figure(figsize=(width,width*aspect_ratio/2))
plt.ion()
ax_mean = plt.subplot(121)
ax_med = plt.subplot(122)
im_mean = cv2.blur(im, (5, 5))
im_med = cv2.medianBlur(im, 5)
plotter_mean = ax_mean.imshow(im_mean, cmap='gray')
plotter_med = ax_med.imshow(im_med, cmap='gray')
ax_med.set_title('Median smoother')
ax_mean.set_title('Mean smoother')
ax_med.axis('off')
ax_mean.axis('off')
plt.tight_layout(pad=0.5)
plt.show()
def update_mean(MeanFilterSize):
    im_mean = cv2.blur(im, (MeanFilterSize, MeanFilterSize))
    plotter_mean.set_data(im_mean)
    fig.canvas.draw_idle()
def update_median(MedianFilterSize):
    im_med = cv2.medianBlur(im, MedianFilterSize)
    plotter_med.set_data(im_med)
    fig.canvas.draw_idle()
interact(update_mean, MeanFilterSize=widgets.IntSlider(min=1,max=55,step=2,value=5, description='Mean filter size',style=style));
interact(update_median, MedianFilterSize=widgets.IntSlider(min=1,max=55,step=2,value=5, description='Median filter size',style=style));

There are more sophisticated smoothers that can be used for denoising, but generally we should be able to get away without them. We can use examples the Bergman smoother and the fast means smoother. If they produce a smoother image, you can select them for further processing.

fig = plt.figure(figsize=(width,width*aspect_ratio/2))
plt.ion()
ax_berg = plt.subplot(121)
ax_cons = plt.subplot(122)
im_bergman = img_as_ubyte(denoise_tv_bregman(im, weight=1.2, channel_axis=None))
im_fm_blur = cv2.fastNlMeansDenoising(im, None, 20, 15, 35)
plotter_berg = ax_berg.imshow(im_bergman, cmap='gray')
plotter_cons = ax_cons.imshow(im_fm_blur, cmap='gray')
ax_berg.set_title('Bergman smoother')
ax_cons.set_title('Fast Means smoother')
ax_berg.axis('off')
ax_cons.axis('off')
plt.tight_layout(pad=0.5)
plt.show()
def update_berg(BergmanFilterSize):
    im_bergman = img_as_ubyte(denoise_tv_bregman(im, weight=BergmanFilterSize, channel_axis=None))
    plotter_berg.set_data(im_bergman)
    fig.canvas.draw_idle()
def update_fastMeans(ConservativeFilterSize,TempWindSize,SearchWindSize):
    im_fm_blur = cv2.fastNlMeansDenoising(im,  None, ConservativeFilterSize, TempWindSize, SearchWindSize)
    plotter_cons.set_data(im_fm_blur)
    fig.canvas.draw_idle()
interact(update_berg, BergmanFilterSize=widgets.FloatSlider(min=0.5, max=5., step=0.1, value=1.2,  description='Bergman filter weight',style=style));
w_slider1 =  widgets.IntSlider(min=5, max=25, step=2, value=15, description='Template window size',style=style) 
w_slider2 = widgets.IntSlider(min=17, max=55, step=2, value=35, description='Search window size',style=style)
h_slider = widgets.FloatSlider(min=5.0, max=50.0, step=0.1, value=20.0, description='Fast means filter weight',style=style)
interact(update_fastMeans, ConservativeFilterSize=h_slider,TempWindSize=w_slider1,SearchWindSize=w_slider2);



After the smoothing is done, select which outputs you wish to process further in the cell below.

list_of_images = [im, im_mean, im_med, im_bergman, im_fm_blur]
list_of_titles = ['Original','Mean smoothed', 'Median smoothed', 'Bergman smoothed','Fast Means smoothed']
checkboxes = [widgets.Checkbox(value=False, description=label) for label in list_of_titles]
output = widgets.VBox(children=checkboxes)
display(output)

## Run the cell below whenever you select or de-select a checkbox in the list above!

    selected_images = []
    selected_titles = []
    for i in range(0, len(checkboxes)):
        if checkboxes[i].value == True:
            selected_titles = selected_titles + [checkboxes[i].description]
            selected_images.append(list_of_images[i])
    print(selected_titles)

# Image erosion
If there are many overlapping eggs in the photo, or they are located closely togerher so that the boundaries are indistinguishable, we can erode the image by convolving it with a kernel - a square matrix of odd size (can take values above 3). In the erosion process, a pixel of the image is set to 1 if all pixels in the kernel surrounding it are also 1. Otherwise, it is set to 0. The erosion process may reduce the area of the eggs on the image, but will create more pronounced boundaries between them The slider below sets the size of the kernel matrix. 

# initial kernel size
initSize = 5
# create kernel
kernel = np.ones((initSize, initSize), np.uint8)
# erode images from the list of images selected above
numCols = 2
numRows = math.ceil(len(list_of_images)/numCols)
fig, axes =  plt.subplots(numRows, numCols, figsize=(width,width*aspect_ratio*numRows/numCols))
plt.ion()
#  loop over the images that were selected above and stored into the list
plotters = []
eroded_titles = []
eroded_images = []
for iImage, image in enumerate(selected_images):
    eroded_titles.append('Eroded ' + selected_titles[iImage])
    ax = axes.flatten()[iImage]
    im_eroded = cv2.erode(255-image, kernel, iterations=1)
    eroded_images.append(im_eroded)
    plotters.append(ax.imshow(im_eroded, cmap='gray'))
    ax.set_title(eroded_titles[iImage])
    ax.axis('off')
if iImage < numRows*2:
    for iEmptyImage in range(iImage+1,numRows*2):
        ax = axes.flatten()[iEmptyImage]
        fig.delaxes(ax)
plt.tight_layout(pad=0.5)
plt.show()    


def update_erode(KernelSize):
    #  update the kernel and run the erosion  
    kernel = np.ones((KernelSize, KernelSize), np.uint8)
    for iImage, image in enumerate(selected_images):
        ax = axes.flatten()[iImage]
        im_eroded = cv2.erode(255-image, kernel, iterations=1)
        eroded_images[iImage] = im_eroded
        plotters[iImage].set_data(im_eroded)
    fig.canvas.draw_idle()
interact(update_erode, KernelSize=widgets.IntSlider(min=3,max=35,step=2,value=initSize, description='Kernel size',style=style));

checkboxes1 = [widgets.Checkbox(value=False, description=label) for label in eroded_titles]
output = widgets.VBox(children=checkboxes1)
display(output)

## Run the cell below whenever you select or de-select a checkbox in the list above!

    selected_images1 = []
    selected_titles1 = []
    for i in range(0, len(checkboxes1)):
        if checkboxes1[i].value == True:
            selected_titles1 = selected_titles1 + [checkboxes1[i].description]
            selected_images1.append(eroded_images[i])
    print(selected_titles1)

numRows = len(selected_titles1)
numCols = 2
blobbed_images = []
fig, axes = plt.subplots(numRows,numCols, figsize=(width,width*aspect_ratio*numRows/numCols))
for iRow, image in enumerate(selected_images1):
    # check the method from stack exchange on thresholding
    thresh = cv2.adaptiveThreshold(im, 225, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 65, 3)
    # apply morphology open then close
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
    blob = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (9,9))
    blob = cv2.morphologyEx(blob, cv2.MORPH_CLOSE, kernel)
    blobbed_images.append(blob)
    axes[iRow,0].imshow(im, cmap='gray')
    axes[iRow,0].set_title(selected_titles1[iRow])
    axes[iRow,0].axis('off')
    axes[iRow,1].imshow(blob, cmap='gray')
    axes[iRow,1].set_title('Blobs')
    axes[iRow,1].axis('off')
plt.tight_layout(pad=0.5)
plt.show()
    

# Contrast adjustment
We can adjust the contrast of the images to make sure there are no over- or underexposed areas in the image. This ensures that the detection algorithm will find blobs everywhere in the image with the same settings.

# create clahe object that defines the matrix
tileSize = 70
clahe = cv2.createCLAHE(clipLimit=5.0, tileGridSize=(tileSize, tileSize))

# Blob detection
Create a detector with the set of parameters. This is the part where we do most of the parameter tuning. We will be usingseveral features of the "blobs" (the eggs) to threshold them from other image artifacts: area size, circularity, convexity, inertia, and color. The image should help to navigate these features. In the image, the values of all features of interes increase from left to right. For example, the colour values closer to 0 represent lighter shades of grey, and values closer to 255 darker shade.
![BlobTest.jpg](attachment:BlobTest.jpg)
By default the thresholding in this detection algorithm is done by colour. You can select other filters for thresholding in the cell below.

# Set up the detector with default parameters:
detector = cv2.SimpleBlobDetector()
# Setup SimpleBlobDetector parameters
list_of_filters = ['Filter by area','Filter by circularity', 'Filter by convexity', 'Filter by inertia']
checkboxes2 = [widgets.Checkbox(value=False, description=label) for label in list_of_filters]
output = widgets.VBox(children=checkboxes2)
display(output)

In the cell below you can change the detector parameter values for each filter. 
## You will have to re-run the cell below every time you change any of the values!

# images to process
images_to_detect = selected_images1
# Turn on filters selected in the checkboxes above
filters = []
for i in range(0, len(checkboxes2)):
    filters = filters + [checkboxes2[i].value]
print(filters)
params = cv2.SimpleBlobDetector_Params()
params.filterByArea, params.filterByCircularity, params.filterByConvexity, params.filterByInertia  = filters
###################################################################################################################
# SET PARAMETERS OF THE DETECTOR HERE
# # The most basic way to detect the blobs is by limiting the hue of the greyscale image.
# # note that the colors run from 0 = white to 255 = black.
params.minThreshold = 10
params.maxThreshold = 500

# Filter by Area. In this case, there is no clear upper or lower limit - the area is defined in pixels and will depend strongly on your image resolution.
params.minArea = 50 # must be >0, but the min area will depend on image resolution
params.maxArea = 200 # must be >minArea, but the max area willwill depend on image resolution

# # Filter by Circularity: 1 - perfect circle, 0 - any shape
params.minCircularity = 0.01
params.maxCircularity = 1
#
# # Filter by Convexity 1 - perfectly convex, 0 - allows for many concave areas
params.minConvexity = 0.01 # can take values from 0 to 1
params.maxConvexity = 1 # can take values from minCovnexity to 1

# # Filter by Inertia - how long is elongated the is ellipse allowed to be
params.minInertiaRatio = 0.01 # can take values from 0 to 1
params.minInertiaRatio = 1 # can take values from minInertiaRatio to 1
# end of changable part
###################################################################################################################
# The code below creates a detector with the parameters we have set in this cell
ver = (cv2.__version__).split('.')
if int(ver[0]) < 3:
    detector = cv2.SimpleBlobDetector(params)
else:
    detector = cv2.SimpleBlobDetector_create(params)
    
#  run the blob detection for all selected images
numCols = 2
numRows = math.ceil(len(images_to_detect)/numCols)
fig, axes =  plt.subplots(numRows, numCols, figsize=(width,width*aspect_ratio*numRows/numCols))
plt.ion()
#  loop over the images that were selected above and stored into the list
for iImage, image in enumerate(images_to_detect):   
    ax = axes.flatten()[iImage]
    image = 255-image
    keypoints = detector.detect(image)
    blank = np.zeros((1, 1)) #    blank = np.zeros((1, 1)) #
    im_with_keypoints = cv2.drawKeypoints(image, keypoints, blank, (255, 165, 0), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    ax.imshow(im_with_keypoints)
    ax.set_title(list_of_titles[iImage])
    ax.set_xlabel('Blob count = '+ str(len(keypoints)))
    # Hide X and Y axes label marks
    ax.xaxis.set_tick_params(labelbottom=False)
    ax.yaxis.set_tick_params(labelleft=False)
    # Hide X and Y axes tick marks
    ax.set_xticks([])
    ax.set_yticks([])
if iImage < numRows*2:
    for iEmptyImage in range(iImage+1,numRows*2):
        ax = axes.flatten()[iEmptyImage]
        fig.delaxes(ax)
plt.tight_layout(pad=0.5)
plt.show()
    
    

